# Introduction to Map/Reduce with Python

In this module we are going to look at how to solve problems using the Map/Reduce paradigm. We will solve for `pi` in possibly the most inefficient but fun way: throwing darts at a dartboard!

## What Is Map/Reduce?

Map/Reduce is a method of distributing processing across several CPUs. It shares concepts from multithreading on CPUs and parallel processing on GPUs but with one major difference: tasks are parallelized across machines in a cluster, and task coordination is such that compute units can be added or removed during a job without affecting the outcome. This means jobs can be scaled as-needed limited only by the availability of more computers.

The basic idea of Map/Reduce is that most tasks can be split into two phases: a data filter/select/project (**map**) phase that is highly parallelizable and an aggregation (**reduce**) phase which is less so. Since the map phase involves the most heavy lifting and often involves loading each record of interest no matter how large the data set, it is spread across a cluster and only summarized or interim results are returned and processed further. The reduce phase involves collecting the raw output from mapping and collating or summarizing that data into values that directly or indirectly answer the question at hand.

### Example

The canonical example is counting words in a set of documents. Assuming the documents look like this:

> the quick brown fox jumps over the lazy dog
> 
> she sells sea shells by the seashore
> 
> how much wood would a woodchuck chuck if a woodchuck could chuck wood

... we want to get something like this:

| word | count |
|---|---|
| a | 2 |
| brown | 1 |
| by | 1 |
| chuck | 2 |
| ... | ... |
| shells | 1 |
| the | 3 |
| wood | 2 |
| woodchuck | 2 |
| would | 1 |

Think for a moment how you might solve this in code:

In [1]:
# rough solution to get the idea

text = '''the quick brown fox jumps over the lazy dog she
sells sea shells by the seashore how much wood would a
woodchuck chuck if a woodchuck could chuck wood'''

counts_per_word = {}
for word in text.split():
    if word not in counts_per_word:
        counts_per_word[word] = 0
    counts_per_word[word] += 1

for word, count in sorted(counts_per_word.items()):
    print(word, count)

('a', 2)
('brown', 1)
('by', 1)
('chuck', 2)
('could', 1)
('dog', 1)
('fox', 1)
('how', 1)
('if', 1)
('jumps', 1)
('lazy', 1)
('much', 1)
('over', 1)
('quick', 1)
('sea', 1)
('seashore', 1)
('sells', 1)
('she', 1)
('shells', 1)
('the', 3)
('wood', 2)
('woodchuck', 2)
('would', 1)


If we were going to spread this across thousands of machines, we have to make a small tweak:

In [2]:
# step 1: emit a 1 for every single word found
word_instances = []
for word in text.split():
    word_instances.append((word, 1))

# note that we see repeated words emitted once for each occurance in the input
word_instances

[('the', 1),
 ('quick', 1),
 ('brown', 1),
 ('fox', 1),
 ('jumps', 1),
 ('over', 1),
 ('the', 1),
 ('lazy', 1),
 ('dog', 1),
 ('she', 1),
 ('sells', 1),
 ('sea', 1),
 ('shells', 1),
 ('by', 1),
 ('the', 1),
 ('seashore', 1),
 ('how', 1),
 ('much', 1),
 ('wood', 1),
 ('would', 1),
 ('a', 1),
 ('woodchuck', 1),
 ('chuck', 1),
 ('if', 1),
 ('a', 1),
 ('woodchuck', 1),
 ('could', 1),
 ('chuck', 1),
 ('wood', 1)]

In [3]:
# step 2: some magic happens (ignore this line of code, as it's
#         implicit in whatever map/reduce framework you use
import itertools
collated_word_instances = [
    (k, [x[1] for x in v])
    for k, v in itertools.groupby(sorted(word_instances), lambda x: x[0])
]

# now we have each word followed by a list of values to be summed
collated_word_instances

[('a', [1, 1]),
 ('brown', [1]),
 ('by', [1]),
 ('chuck', [1, 1]),
 ('could', [1]),
 ('dog', [1]),
 ('fox', [1]),
 ('how', [1]),
 ('if', [1]),
 ('jumps', [1]),
 ('lazy', [1]),
 ('much', [1]),
 ('over', [1]),
 ('quick', [1]),
 ('sea', [1]),
 ('seashore', [1]),
 ('sells', [1]),
 ('she', [1]),
 ('shells', [1]),
 ('the', [1, 1, 1]),
 ('wood', [1, 1]),
 ('woodchuck', [1, 1]),
 ('would', [1])]

In [4]:
# step 3: for each word, sum the counts
word_counts = {}
for word, counts in collated_word_instances:
    word_counts[word] = sum(counts)

word_counts.items()

[('over', 1),
 ('chuck', 2),
 ('seashore', 1),
 ('sea', 1),
 ('sells', 1),
 ('if', 1),
 ('would', 1),
 ('shells', 1),
 ('fox', 1),
 ('how', 1),
 ('much', 1),
 ('woodchuck', 2),
 ('brown', 1),
 ('lazy', 1),
 ('jumps', 1),
 ('by', 1),
 ('a', 2),
 ('wood', 2),
 ('could', 1),
 ('dog', 1),
 ('she', 1),
 ('quick', 1),
 ('the', 3)]

While this code is less efficient than the previous solution, it has the characteristic that phase 1 can be run on separate documents by different computers and the full output can be sent to a single computer for summing and reporting the final output in phase 3.

Phase 2 can be quite complex: there's data collation and orchestrating the return of intermediate values to the reducer(s). Also, at scale there would be a lot of intermediate processing of data to greatly reduce the amount of data that actually has to be sent back to the server putting the final answer together, but that's beyond the scope of this lesson. Luckily, you can treat phase 2 as a "magic step" and still use the paradigm successfully.

<img src="./step2.gif" />

Putting it all together into a map phase and a reduce phase (and trusting the framework handles the collation phase for us):

```python
def mapfn(key, value):
    '''
    receive documents and output individual word count tuples
       
    key: some kind of identifier, like a path to the source file
    value: the body of text indicated by the key
    '''
    for word in value.split():
        yield (word, 1)

def reducefn(key, value):
    '''
    sum the counts for each word
       
    key: the word
    value: the list of counts emitted by multiple mapfn calls
    '''
    yield key, sum(value)
```

Now our entire algorithm can be expressed in 2 functions for a total of five lines of code to get the same `word_counts` list generated previously.

## Computing Pi

As has been aluded to earlier, pi can be computed by throwing darts at a dartboard. More specifically, if you randomly put points in a square of dimension `r` and count the number of points that are inside a radius of `r` from the origin vs the total number of points generated, you can compute pi as:

```python
pi_approximation = points_within_one_radius / total_points
```

For a comprehensive analysis of why this is true, see [Estimating the value of Pi using Monte Carlo](https://www.geeksforgeeks.org/estimating-value-pi-using-monte-carlo/) and also [Wikipedia's description of the Monte Carlo method](https://en.wikipedia.org/wiki/Monte_Carlo_method).

### Naive method

The naive method for computing pi in this manner is to just generate a whole bunch of random points and sum up the totals:

In [5]:
import math
import random
import uuid

random.seed(uuid.uuid4())
total = 100000000

print('started computing...')
inside = sum(math.sqrt(random.random() ** 2 + random.random() ** 2) <= 1. for i in range(total))

# multiply by 4 because our "square in a circle" is only one quadrant
4. * inside / total

started computing...


3.14172712

It takes about 30 seconds for my computer to compute this for 100,000,000 points, so it would take roughly an hour to compute ten billion points. We want to go faster!

### Map

A map function that generates `k` random points and counts how many are inside vs. outside one radius from the origin, returning a key of "totals" and a value of (num_inside, total_count):

```python
def mapfn(key, value):
    '''compute `value` random points and return count of inside radius and total points'''
    random.seed(uuid.uuid4())
    inside = sum(
        math.sqrt(random.random() ** 2 + random.random() ** 2) < 1.
        for i in xrange(value)
    )
    yield 'totals', (inside, value)
```

Note that this function will be run many times on many computers, generating a very long list of counts (one for each map task).

### Reduce

A reduce function that sums up the totals computed by all the map functions:

```python
def reducefn(key, value):
    inside = 0
    total = 0
    for v in value:
        inside += v[0]
        total += v[1]
    return inside, total
```

The final totals that the reduce function emits are the sum of *all* points inside one radius and the sum of *all* darts thrown.

Putting that into our earlier formula, we get:

```python
pi_approximation = 4.0 * inside / total
```

## Demo

Let's do this! I'm going to ask all of you to help me compute pi. We are going to request 1000 map tasks, each throwing 10,000,000 darts. That's *ten billion* darts we are going to throw, which should give us a value for pi accurate to about six or seven digits.

The code defines a map function and a reduce function, and then it starts up the server. It will simply sit and wait for workers to connect:

```bash
$ python -m pi 
data: {1: 10000000, 2: 10000000, 3: 10000000, ..., 998: 10000000, 999: 10000000, 1000: 10000000}
waiting for workers...
```

When a worker connects, the server sends it the function to execute followed by tasks one at a time as the worker reports that it is ready for a task.


```bash
shell 1> python -m mincemeat -p pass localhost
mapfn: 1, 10000000
mapfn: 2, 10000000
mapfn: 3, 10000000
...
```

I can start a second worker, and it will pick up whatever task is next and then the two will basically split the work:

```bash
shell 2> python -m mincemeat -p pass localhost
mapfn: 4, 10000000
mapfn: 6, 10000000
mapfn: 8, 10000000
...
```

Once all the tasks have been completed (regardless of which worker completed each task), the server then sends out the single reduce job to one of the workers. The server gets back the result and prints a final answer to the screen.

```bash
results: {'totals': (78546246, 100000000)}
totals: 78546246 inside, 100000000 total, pi ~= 3.14184984
```

See the [complete source code](./pi.py)

When I run this on my 4-processor laptop with 4 task runners running in parallel, it takes 17.9 minutes to complete.

For you to help me run it:

1. install or download `mincemeat`
	 * `pip install mincemeat` inside a virtual environment
	 * ... or just download `mincemeat.py` from [https://bit.ly/2JVFIqR](https://bit.ly/2JVFIqR)
1. once my server is started, run mincemeat: `python -m mincemeat -p pass <server address>`
    * note that `mincemeat` only works with python 2... you might need to specify the python version when running the command: `python2 -m mincemeat -p pass <server address>`
    * note that you can start multiple workers, but your computer will only be efficient running no more workers than you have CPU cores (this particular algorithm is CPU-bound, so context switching within the CPU far outweighs any benefit gained by trying to run additional workers)
    * watch as your computer shows job numbers it is running

Ready, go!